In [ ]:
# from ultralytics.data.converter import convert_coco

# convert_coco(labels_dir="/home/dhw/yyc_workspace/yolo_splited/train/images")

In [ ]:
import os
import json
import cv2
from datetime import datetime

from tqdm import tqdm

def yolo_to_coco(yolo_root_dir, output_path, class_names):
    """
    将YOLO格式数据集转换为COCO格式
    
    Args:
        yolo_root_dir: YOLO数据集根目录，包含images和labels文件夹
        output_path: 输出的COCO格式json文件路径
        class_names: 类别名称列表
    """
    
    images_dir = os.path.join(yolo_root_dir, 'images')
    labels_dir = os.path.join(yolo_root_dir, 'labels')
    
    # 初始化COCO格式数据结构
    coco_data = {
        "info": {
            "description": "YOLO to COCO conversion",
            "version": "1.0",
            "year": datetime.now().year,
            "date_created": datetime.now().isoformat()
        },
        "licenses": [],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    # 添加类别信息
    for i, class_name in enumerate(class_names):
        coco_data["categories"].append({
            "id": i+1,
            "name": class_name,
            "supercategory": ""
        })
    
    annotation_id = 0
    
    # 遍历images文件夹中的图片
    for image_file in tqdm(os.listdir(images_dir)):
        if not image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            continue
            
        image_path = os.path.join(images_dir, image_file)
        image_name = os.path.splitext(image_file)[0]
        label_file = image_name + '.txt'
        label_path = os.path.join(labels_dir, label_file)
        
        # 读取图片获取宽高
        image = cv2.imread(image_path)
        if image is None:
            print(f"无法读取图片: {image_path}")
            continue
            
        height, width = image.shape[:2]
        image_id = len(coco_data["images"])
        
        # 添加图片信息
        coco_data["images"].append({
            "id": image_id,
            "width": width,
            "height": height,
            "file_name": image_file
        })
        
        # 如果对应的标签文件不存在，跳过
        if not os.path.exists(label_path):
            print(f"标签文件不存在: {label_path}")
            continue
        
        # 读取YOLO格式标签文件
        with open(label_path, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            parts = line.split()
            if len(parts) != 5:
                print(f"标签格式错误: {line}")
                continue
            
            class_id = int(parts[0]) + 1
            x_center = float(parts[1])
            y_center = float(parts[2])
            bbox_width = float(parts[3])
            bbox_height = float(parts[4])
            
            # 将YOLO格式(相对坐标)转换为COCO格式(绝对坐标)
            x_min = (x_center - bbox_width / 2) * width
            y_min = (y_center - bbox_height / 2) * height
            bbox_width_abs = bbox_width * width
            bbox_height_abs = bbox_height * height
            
            # 确保坐标在图片范围内
            x_min = max(0, x_min)
            y_min = max(0, y_min)
            bbox_width_abs = min(bbox_width_abs, width - x_min)
            bbox_height_abs = min(bbox_height_abs, height - y_min)
            
            area = bbox_width_abs * bbox_height_abs
            
            # 添加标注信息
            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "bbox": [x_min, y_min, bbox_width_abs, bbox_height_abs],
                "area": area,
                "iscrowd": 0
            })
            
            annotation_id += 1
    
    # 保存COCO格式json文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(coco_data, f, indent=2, ensure_ascii=False)
    
    print(f"转换完成！")
    print(f"图片数量: {len(coco_data['images'])}")
    print(f"标注数量: {len(coco_data['annotations'])}")
    print(f"类别数量: {len(coco_data['categories'])}")
    print(f"输出文件: {output_path}")
    

if __name__ == "__main__":
    # 示例用法
    mode = "train"
    yolo_dir = f"/home/dhw/yyc_workspace/dataset_coco/{mode}"  # 包含images和labels文件夹的目录
    output_path = f"/home/dhw/yyc_workspace/dataset_coco/anno_{mode}.json"
    class_names = ["ship", "helicopter", "UAV"]  # 根据您的数据集类别修改
    
    yolo_to_coco(yolo_dir, output_path, class_names)